In [21]:
#VSE SKUPAJ-----CV verzija 28.3.2021---------------IMAGES--------------------For loop for k-----------------------------
#To add new metric: https://stackoverflow.com/questions/21052509/sklearn-knn-usage-with-a-user-defined-metric
#deluje----ok
#1) load data IMAGES	2) defin PM	4) lmnn	4) knn	5) CV


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score   #for crossvalidation
from sklearn.model_selection import StratifiedKFold

from pylmnn import LargeMarginNearestNeighbor as LMNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score    #for confusion matrix
from sklearn import datasets, neighbors, metrics   # Import datasets, classifiers and performance metrics

from sklearn.preprocessing import Normalizer	#for normalization

from sklearn.neighbors import DistanceMetric
from sklearn.preprocessing import StandardScaler    ##for standardization-Gaussian with zero mean and unit variance.



In [22]:
train_df = pd.read_csv('cbclAutores150.csv')
X = np.array(train_df.iloc[:, :-1])	#all rows & all column except last one
#print(train_data)
y = np.array(train_df.iloc[:, -1])   #all rows & only last column

X.shape
print(f"Mn X {X.shape}")


Mn X (3192, 1024)


In [23]:
#Normalization dataset
transformer = Normalizer().fit(X)
X = transformer.transform(X)

#print(X)
#print(y)

In [24]:
#Before making any actual predictions, it is always a good practice to scale the features so that all of them can be uniformly evaluated = standardization
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)


In [25]:
#Define functions to compute the value of the distance metrics: Poincare; ŠE NI OK
#Define functions to compute the value of the distance metrics: Poincare; ŠE NI OK
#@staticmethod
def poin_distance(row1, row2):
    distance = 0.0
    d = 0.0
    norm1 = 0.0
    norm2 = 0.0
    for i in range(len(row1)):		#for i in range(len(row1)-1):
        d += (row1[i] - row2[i])**2
        norm1 += (row1[i] * row1[i])
        norm2 += (row2[i] * row2[i])
        m = 1 + 2*d/((1-norm1)*(1-norm2))
        distance = np.log(m + np.sqrt(m*m - 1))
        return(distance)


In [26]:
# LMNN: algorithm for metric learning---------------------after cross validation
# Set up the hyperparameters
k_train, k_test, max_iter = 3, 3, 180	#n_components=X.shape[1], za bpp:640

# Instantiate the metric learner
lmnn = LMNN(n_neighbors=k_train, max_iter=max_iter)

# Train the metric learner
lmnn.fit(X,y)


LargeMarginNearestNeighbor(callback=None, impostor_store='auto', init='pca',
                           max_impostors=500000, max_iter=180,
                           n_components=None, n_jobs=1, n_neighbors=3,
                           neighbors_params=None, random_state=None,
                           store_opt_result=False, tol=1e-05, verbose=0,
                           warm_start=False, weight_push_loss=0.5)

In [ ]:
# Create a classifier: a knn with PM
for i in range(1,11):
    knn = neighbors.KNeighborsClassifier(n_neighbors=i, leaf_size=3, algorithm='auto', metric='pyfunc', metric_params={"func":poin_distance}, n_jobs=4) 	#to add new metric: metric='pyfunc'
	#for testing
	#knn = neighbors.KNeighborsClassifier(n_neighbors=5, algorithm='brute', metric='euclidean')    #algorithm='ball_tree' - for a custom metric

	# Learn the digits/iris on the train subset - fit the model
    knn.fit(lmnn.transform(X),y)

	#cross validation-------------------where is this------------after knn
    scores = cross_val_score(knn, lmnn.transform(X), y, cv=StratifiedKFold(10), n_jobs=4)	#rez. se ne spremeijo, če dam tuki lmnn.transform(X)--POPRAVKI

	# Predict the value of the digit on the test subset---from https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html#sphx-glr-auto-examples-classification-plot-digits-classification-py
    predicted = knn.predict(lmnn.transform(X))
    lmnn_acc = knn.score(lmnn.transform(X), y)
    #for results
    #print(f"Classification report for classifier {knn}:\n"
    #      f"{metrics.classification_report(y, predicted)}\n")   

    # Print out confusion matrix
    cmat = confusion_matrix(y, predicted)	#print(cmat)

    #print('Accuracy Rate: {}'.format(np.divide(np.sum([cmat[0,0],cmat[1,1]]),np.sum(cmat))))
    #print('Misclassification Rate: {}'.format(np.divide(np.sum([cmat[0,1],cmat[1,0]]),np.sum(cmat))))

    #print(cmat)

    #print('Diagonal (sum): ', np.trace(cmat))
    #print("All Instances:", np.sum(cmat))
    print("Correctly Classified Instances:", np.trace(cmat)/np.sum(cmat))
    print('Number of neighbors: ', i)


Correctly Classified Instances: 0.9890350877192983
Number of neighbors:  1
Correctly Classified Instances: 0.9514411027568922
Number of neighbors:  2
